### Combining GDP data with the jobs report

Brian Dew

Updated: September 20, 2020

----

Notes:

The basic idea here is to use the BEA population estimates plus CPS employment rate and hours worked trends to estimate GDP per hour of work. BLS does this process for its productivity and costs report, using much more comprehensive data, however this approximation has proven decent over time.

The biggest challenge is correctly estimating the average hours worked. All of the published measures are either too-low-frequency or don't have a broad enough definition of workers. I want to capture all workers, regardless of full-time or part-time status or of whether they work for the private sector. Also want to capture second and third jobs. As a result, I selected total actual hours worked from the CPS microdata, specifically finding the trend using x13as with default settings. Hours worked from the CPS microdata have issues around holidays falling in the reference week, and also do not capture hours worked for some important categories of labor such as self-employed persons.

In [1]:
%matplotlib inline
import sys
sys.path.append('../src')

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

qtrs = {1: 1, 2: 1, 3: 1, 4: 2, 5:2, 6:2, 7:3, 8:3, 9:3, 10:4, 11:4, 12:4}

In [2]:
# CPS-based estimated employment rate and average actual hours
avghrs = lambda x: np.average(x.HRSACTT.replace(-1, 0), weights=x.WGT)
aah, epop = {}, {}
cols = ['HRSACTT', 'LFS', 'MONTH', 'AGE']
for year in range(1989, 2025):
    wgt = 'PWSSWGT' if year >= 1998 else 'BASICWGT'
    df = pd.read_feather(cps_dir / f'cps{year}.ft', 
                         columns=cols + [wgt]).rename({wgt: 'WGT'}, axis=1)
    emp = df.query('LFS == "Employed" and AGE >= 15')
    ah = (emp.groupby('MONTH').apply(avghrs))
    aah.update({pd.to_datetime(f'{year}-{month}-01'): value 
                for month, value in list(zip(ah.index, ah.values))})
    ep = (emp.groupby('MONTH').WGT.sum() / df.groupby('MONTH').WGT.sum()) * 100
    epop.update({pd.to_datetime(f'{year}-{month}-01'): value 
                 for month, value in list(zip(ep.index, ep.values))})

In [3]:
# Seasonally-adjust monthly data and save
res = pd.DataFrame({'EPOP': epop, 'AAH': aah})
pop = (pd.read_csv(data_dir/'nipa20600.csv', index_col='date', 
                  parse_dates=True))
res['POP'] = pop['B230RC'] / 1000

res['EPOP_sa'] = x13_arima_analysis(res['EPOP']).seasadj
res['AAH_trend'] = x13_arima_analysis(res['AAH']).trend
res['AAH_sa'] = x13_arima_analysis(res['AAH']).seasadj

# Fill future population data from Census
popm = (pd.read_csv(data_dir/'pop_est_bea.csv', index_col='date', 
                    parse_dates=True).divide(1000).round(-1).divide(1000))
pop = pd.concat([res['POP'].dropna(), popm['POP']])
pop = pop[~pop.index.duplicated(keep='last')]
res = pd.concat([res.drop('POP', axis=1), pop], axis=1)

# Handle cases where no population estimate is available for December, yet
if f'{res.index[-1].year}-12-01' not in res.index:
    poplt = ((res.POP.dropna().pct_change().iloc[-1] + 1) 
             * res.loc[f'{res.index[-1].year}-11-01', 'POP'])
    res.loc[f'{res.index[-1].year}-12-01', 'POP'] = poplt

# Estimate total hours
res['EMP_sa'] = ((res['EPOP_sa'] / 100) * res['POP']) / 1_000
res['TOT_HRS'] = (res['EMP_sa'] * res['AAH_trend'] * 52)

res.to_csv(data_dir / 'gdpjobslvl_mo.csv', index_label='date')

/home/brian/miniconda3/lib/python3.8/site-packages/statsmodels/tsa/x13.py:203: X13Warning: WARNING: At least one visually significant trading day peak has been
          found in the estimated spectrum of the regARIMA residuals.
  
          found in one or more of the estimated spectra.
  warn(errors, X13Warning)
/home/brian/miniconda3/lib/python3.8/site-packages/statsmodels/tsa/x13.py:203: X13Warning: WARNING: At least one visually significant trading day peak has been
          found in the estimated spectrum of the regARIMA residuals.
  
          found in one or more of the estimated spectra.
  warn(errors, X13Warning)


In [4]:
res = pd.read_csv(data_dir / 'gdpjobslvl_mo.csv', index_col='date', 
                  parse_dates=True).dropna()
ltpop = value_text(res['POP'].iloc[-1], style='plain', 
                   ptype='million', digits=0, round_adj=True)
ltdt = dtxt(res.index[-1])['mon1']
chval = res.dropna()['POP'].iloc[-1] - res['POP'].iloc[0]

chtxt = value_text(chval, style='plain', casual=True,
                   ptype='million', digits=0, round_adj=True)

# Population Growth rates
popgr = growth_rate_monthly(res['POP'])
ltpopgr = value_text(popgr.iloc[-1], style='plain')
prpopgr = value_text(popgr.mean(), style='plain')

# Population text
cl = c_line('lime!90!green')
text = (f'First, the US \\textbf{{population}} \index{{population}} '+
        f'is {ltpop}, as of {ltdt} {cl}, '+
        f'an increase of {chtxt} since 1989. Since 1989, the '+
        f'population has grown at an average annual rate of {prpopgr}; '+
        f'the current rate is around {ltpopgr}.')
write_txt(text_dir / 'gdpjobspop.txt', text)
print(text, '\n')

# Epop text 
ltdt = dtxt(res.index[-1])['mon1']
ltval = value_text(res['EPOP_sa'].iloc[-1], style='plain')
mndiff = res['EPOP_sa'].mean() - res['EPOP_sa'].iloc[-1]
difftxt = value_text(-mndiff, style='above_below', ptype='pp')

text = (f'The current rate is {difftxt} '+
        'the 30-year average.')
write_txt(text_dir / 'gdpjobsepop.txt', text)
print(text, '\n')

# Average workweek text
ltdt = dtxt(res.index[-1])['mon1']
lthrs = res['AAH_trend'].iloc[-1].round(1)

text = (f'In {ltdt}, the average workweek is {lthrs} hours.')
write_txt(text_dir / 'gdpjobsaah.txt', text)
print(text, '\n')

# End nodes
colors = {'EPOP_sa': 'green!30!teal!90!black', 'AAH_trend': 'blue', 
          'POP': 'lime!90!green', 'TOT_HRS': 'orange'}
for name, color in colors.items():
    adj = node_adj(res[name].to_frame())[name]
    node = end_node(res[name], color, digits=1, offset=adj,
                    date='m', full_year=True)
    write_txt(text_dir / f'gdpjobnode_{name}.txt', node)

# Total hours worked text
pcdt = res['TOT_HRS'].loc['2019':'2020'].idxmax()
pcdtxt = dtxt(pcdt)['mon1']
ch89txt = value_text(growth_rate_monthly(res['TOT_HRS']).mean(), 
                     adj='avg_ann')
chpc = ((res['TOT_HRS'].iloc[-1] / 
         res.loc[pcdt, 'TOT_HRS'].mean()) - 1) * 100
chpctxt = value_text(chpc, adj='total')

text = (f'Since 1989, total hours worked have {ch89txt}. '+
        f'Since the pre-pandemic peak in {pcdtxt}, hours '+
        f'worked have {chpctxt}. ')
write_txt(text_dir / 'gdpjobstothrs.txt', text)
print(text)

First, the US \textbf{population} \index{population} is nearly 337 million, as of April 2024 (see {\color{lime!90!green}\textbf{---}}), an increase of 90 million since 1989. Since 1989, the population has grown at an average annual rate of 0.9 percent; the current rate is around 0.5 percent. 

The current rate is 1.1 percentage points above the 30-year average. 

In April 2024, the average workweek is 36.9 hours. 

Since 1989, total hours worked have increased at an average annualized rate of 1.3 percent. Since the pre-pandemic peak in October 2019, hours worked have decreased by a total of 0.9 percent. 


### Labor Productivity

See technical note about sectors excluded by nonfarm business productivity: https://www.bls.gov/news.release/prod2.tn.htm


In [5]:
res = pd.read_csv(data_dir / 'gdpjobslvl_mo.csv', index_col='date', 
                  parse_dates=True)

data = res.resample('QS').mean()
cd = nipa_df(retrieve_table('T10105')['Data'], ['A191RC'])['A191RC'].iloc[-1]
rgdp = nipa_df(retrieve_table('T10106')['Data'], ['A191RX'])
gdp = rgdp / rgdp.iloc[-1] * cd
data['GDP'] = gdp['A191RX']
data['LPROD'] = data['GDP'] / data['TOT_HRS']

# Official estimate for nonfarm businesses from BLS
data['LPROD_BLS'] = fred_df2('OPHNFB', start='1988')
data['HRS_BLS'] = fred_df2('HOANBS', start='1988')

# Convert to indices
for i in ['LPROD', 'LPROD_BLS','LPROD','TOT_HRS', 'GDP', 
          'POP', 'EPOP_sa', 'AAH_trend']:
    data[f'{i}_idx'] = (data[i] / data.loc['2000-01-01', i]) * 100
    
data.to_csv(data_dir / 'gdpjobslvl.csv', index_label='date')    

In [6]:
# Store calculations for key series and date ranges
srs = ['LPROD', 'GDP', 'TOT_HRS']

ltdt = data.dropna().index[-1]
dts = [('1989-01-01', '1999-01-01'),
       ('1999-01-01', '2009-01-01'),
       ('2009-01-01', '2019-01-01'),
       ('1989-01-01', '2019-10-01'), 
       ('2019-10-01', ltdt)]

d = {}
for sr, (dt1, dt2) in itertools.product(srs, dts):
    t1 = dt1[2:4]
    t2 = dt2[2:4] if isinstance(dt2, str) else 'LT'
    n = f'{sr}{t1}{t2}'
    s = data.loc[dt1:dt2, sr]
    r = cagr(s)
    d[f'cagr_{n}'] = r
    d[n] = ((s.iloc[-1] / s.iloc[0]) - 1) * 100
    
ch8999 = value_text(d['LPROD8999'])
ch9909 = value_text(d['LPROD9909'])
ch0919 = value_text(d['LPROD0919'])

ltgr = value_text(d['cagr_LPROD8919'], style='plain')
ltgdpgr = value_text(d['cagr_GDP8919'], style='plain')
lthrsgr = value_text(d['cagr_TOT_HRS8919'])
rgr = value_text(d['cagr_LPROD19LT'], style='plain')
rgdpgr = value_text(d['cagr_GDP19LT'], style='plain')
rhrsgr = value_text(d['cagr_TOT_HRS19LT'], style='increase_of', 
                    adj='average', threshold=0.1)

text = ('Labor productivity has increased substantially over the '+
        'long term. From 1989 to 1999, labor productivity for the '+
        f'total economy {ch8999}. From 1999 to 2009, labor '+
        f'productivity {ch9909}, and from 2009 to 2019, labor '+
        f'productivity {ch0919}.\n\nFrom 1989 to 2019, total '+
        f'economy productivity growth averaged {ltgr} per year; '+
        f'GDP growth averaged {ltgdpgr} per year and work hours '+
        f'{lthrsgr} per year. Since 2019, total economy labor '+
        f'productivity growth averages {rgr} per year, with average '+
        f'GDP growth of {rgdpgr} and {rhrsgr} in work hours.')
write_txt(text_dir / 'gdpjobs_lprod.txt', text)
print(text)

Labor productivity has increased substantially over the long term. From 1989 to 1999, labor productivity for the total economy increased 17.2 percent. From 1999 to 2009, labor productivity increased 18.9 percent, and from 2009 to 2019, labor productivity increased 11.0 percent.

From 1989 to 2019, total economy productivity growth averaged 1.5 percent per year; GDP growth averaged 2.5 percent per year and work hours increased one percent per year. Since 2019, total economy labor productivity growth averages 2.2 percent per year, with average GDP growth of two percent and an average decrease of 0.2 percent in work hours.


### Contributions to growth

In [7]:
# Quarterly contribution to growth
srs = {'EPOP_sa': 'epop_contr', 'POP': 'pop_contr', 
       'AAH_trend': 'hours_contr', 'lprod': 'prod', 
       'GDP': 'gdp'}
df = data.copy()[['GDP', 'POP', 'EPOP_sa', 'AAH_trend']].dropna()
df['hh_inp'] = df['POP'] * df['EPOP_sa'] * df['AAH_trend']
df['lprod'] = df['GDP'] / df['hh_inp']
c = ((((df.pct_change() + 1) ** 4) - 1) * 100).rename(srs, axis=1).dropna()
c.to_csv(data_dir / 'gdpjobs.csv', index_label='date')

# Annual GDP Growth Contribution
dfa = (c.groupby(c.index.year).filter(lambda x: len(x) == 4)
            .resample('AS').mean())
dfa.to_csv(data_dir / 'gdpjobsa.csv', index_label='date')

# Text
prdt = '2019-10-01'
prdate = dtxt(pd.to_datetime(prdt))['qtr1']
prdate2 = dtxt(pd.to_datetime(prdt))['qtr2']
datetxt = dtxt(c.index[-1])['qtr1']
lt = c.iloc[-1]
pr = c.loc[prdt]

# Change default settings
def vt(value, style='contribution', casual=False):
    return value_text(value, style=style, ptype='pp', 
                      digits=2, casual=casual, threshold=0.02)

poplt = vt(lt['pop_contr'], style='contribution_to')
poppr = vt(pr['pop_contr'], casual=True)
emplt = vt(lt['epop_contr'])
emppr = vt(pr['epop_contr'], casual=True)
hrslt = vt(lt['hours_contr'], style='contribution_to', casual=True)
hrspr = vt(pr['hours_contr'], casual=True)
prodlt = vt(lt['prod'], style='contribution_to')
prodpr = vt(pr['prod'], style='contribution_of', casual=True)

text = (f'In {datetxt}, population growth {poplt} annualized GDP growth, '+
        f'and, for comparison, {poppr} in {prdate}. Changes in the '+
        f'employed share of the population {emplt} in the latest quarter, '+
        f'and {emppr} in {prdate2}. Changes in average '+
        f'hours worked {hrslt} GDP growth in the latest quarter and {hrspr} '+
        f'in {prdate}. Lastly, productivity {prodlt} GDP growth in {datetxt}, '+
        f'compared to {prodpr} in {prdate}.')
write_txt(text_dir / 'gdpjobsch.txt', text)
print(text)

In 2024 Q2, population growth contributed 0.46 percentage point to annualized GDP growth, and, for comparison, added 0.58 percentage point in 2019 Q4. Changes in the employed share of the population contributed 0.96 percentage point in the latest quarter, and added 1.62 percentage points in the fourth quarter of 2019. Changes in average hours worked subtracted 0.13 percentage point from GDP growth in the latest quarter and added 0.36 percentage point in 2019 Q4. Lastly, productivity contributed 1.53 percentage points to GDP growth in 2024 Q2, compared to virtually no addition in 2019 Q4.


In [8]:
# Summary of latest year text
sl = {'pop_contr': 'population', 'epop_contr': 'employment', 
      'hours_contr': 'hours worked', 'prod': 'labor productivity'}
ltyval = dfa[sl.keys()].iloc[-1].rename(sl)
gctext, _ = gc_desc(ltyval, 2, 5)
st = 'is the result of'
if gctext.count(st) > 1:
    gctext = gctext.split(st)[0] + st + gctext.split(st)[2]
    
epopnm = ('the higher employment rate' if dfa.epop_contr.iloc[-1] > 0.4 
          else 'the lower employment rate' if dfa.epop_contr.iloc[-1] < -0.4
          else 'the employment rate')
hoursnm = ('longer average workweeks' if dfa.hours_contr.iloc[-1] > 0.4 
          else 'the drop in hours worked' if dfa.hours_contr.iloc[-1] < -0.4
          else 'hours worked')
rn = {'employment': epopnm,
      'population': 'population growth',
      'hours worked': hoursnm}
ltyval = ltyval.rename(rn).sort_values()
ltyr = dfa.index[-1].year
cat1 = ltyval.index[-1].capitalize()
cv1 = value_text(ltyval.iloc[-1], 'contribution', casual=True, ptype='pp')
cat2 = ltyval.index[-2]
cv2 = value_text(ltyval.iloc[-2], 'contribution', casual=True, ptype='point')
cat3 = ltyval.index[0]
cv3 = value_text(ltyval.iloc[0], 'contribution_to', casual=True, ptype='point')

text = (f'In the latest full year of data, covering {ltyr}, {gctext} '+
        f'{cat1} {cv1}, {cat2} {cv2}, and {cat3} {cv3} overall growth. ')
write_txt(text_dir / 'gdpjobsch_ann.txt', text)
print(text)

In the latest full year of data, covering 2023, growth is relatively broad-based. The main contribution is an increase in labor productivity and employment. Labor productivity added 2.3 percentage points, the higher employment rate added 0.7 point, and hours worked subtracted 0.3 point from overall growth. 
